## Defining the Feed Forward Neural Network class

In [1]:
'''
Name : Sukannya Purkayastha
Roll No: 18CS71P06

Assignment 1b
'''

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

df= pd.read_csv('Iris_Data.csv')
df.head()

'''
You will not import any other library other than these provided.

We provide the iris_dataset for the classification task
There are 4 dependent variables columns(1-4).
The last column (category of the flower) is what we wish to predict

The first part of this task is similar to assignment 1 a
'''
# reads the file and stores in 2 numpy arrays.
# X has the input features and Y has the output value in numpy array

X = df.iloc[:,:-1].values
Y = df.iloc[:,-1].values

rows,cols= df.shape #?, #? 
# how to get the number of rows and columns in the dataset.
# Rows correspond to the number of input instances, columns correspond to the feature of an input

print(rows,cols)

np.random.seed(42) # to ensure that the same seed is generated

# write code to shuffle the dataset

def shuffle_dataset(X,Y):
    np_combined_array=np.column_stack((X,Y))
    np_combined_array=np.random.permutation(np_combined_array)
    X=np_combined_array[:,:-1]
    Y=np_combined_array[:,-1]
    return X,Y
    
    pass

X,Y = shuffle_dataset(X,Y)
training_size = int(0.8*rows)

X_train = X[:training_size]
y_train = Y[:training_size]
#print(y_train.type)
b = np.zeros((y_train.shape[0], 3)).astype(int)
b[np.arange(y_train.shape[0]), y_train.astype(int)] = 1
y_train=b
X_test = X[training_size:]
y_test = Y[training_size:]
b = np.zeros((y_test.shape[0], 3)).astype(int)
b[np.arange(y_test.shape[0]), y_test.astype(int)] = 1
y_test=b

150 5


In [2]:
class NeuralNetwork(object):
    
    def __init__(self, input_no, hidden_no, output_no ):
        '''
            Initialize the Neural network model 
            Args:
                input_no : no of input features (no of cols)
                hidden_no: no of hidden nodes in the model
                output_no: no of categories our model can classify
                      
        
        '''
        self.h=  np.zeros(hidden_no-1)# Initialize the hidden layer with zero ?
        self.w1= np.random.uniform(0,0.01, size=(input_no-1,hidden_no))# Initialize the weights from the input to the hidden layer uniformly with values between 0 and 0.01  ?
        self.b1= np.random.uniform(0,0.01, size=(1,hidden_no))# Initialize the biases uniformly with values between 0 and 0.01 equal to the number of hidden nodes  ?
        self.w2= np.random.uniform(0,0.01, size=(hidden_no,output_no))# Initialize the weights from the hidden layer to the output uniformly with values between 0 and 0.01 ? 
        self.b2= np.random.uniform(0,0.01, size=(1,output_no)) # Initialize the biases uniformly with values between 0 and 0.01 equal to the number of output categories ?
        pass

    
    def forward(self, x):
        '''
            Do a forward pass on the NN model 
            Args: 
                x : Input feature matrix 
                
            Return:
                y_pred : list of predicted probabilities of x

                h= relu(w1.x+b1) 
                y_pred = softmax(w2.h+b2)
                
        
        '''
        x=np.matrix(x)
        self.w1=np.matrix(self.w1)
        self.h = np.add(x*self.w1, np.array(self.b1))
        self.h[self.h<0]=0   #relu
        output_values=np.add((np.matrix(self.h)*np.matrix(self.w2)), np.array(self.b2))
        ps = np.empty(output_values.shape)
        y_pred=[]
        for i in range(output_values.shape[0]):
            ps[i,:]  = np.exp(output_values[i,:])
            ps[i,:] /= np.sum(ps[i,:])
            #print(ps)
            y_pred.append(ps[i,:])
        
        #print(ps)
        
        
        return y_pred
        pass
    
    def backward(self, x, y_train, y_pred, lr):
        
        '''
            Do a backward pass on the NN model. 
            Computes all gradients and updates the parameters w1, b1, w2, b2
            
            Args:
                x: input matrix X 
                y_train: actual category of the feature/ data point
                y_pred: predicted probabilities of the categories obtained during forward pass
                lr: learning rate        
        '''
        output_error_signal = (y_pred - y_train) / y_train.shape[0]
        error_signal_hidden = np.dot(output_error_signal, self.w2.T) 
        error_signal_hidden[self.h <= 0] = 0
        gradient_layer2_weights = np.dot(self.h.T, output_error_signal)
        gradient_layer2_bias = np.sum(output_error_signal, axis = 0, keepdims = True)
        gradient_layer1_weights = np.dot(x.T, error_signal_hidden)
        gradient_layer1_bias = np.sum(error_signal_hidden, axis = 0, keepdims = True)
            
        self.w2-=(gradient_layer2_weights*lr)
        self.b2-=(gradient_layer2_bias*lr)
        self.w1-=(gradient_layer1_weights*lr)
        self.b1-=(gradient_layer1_bias*lr)
        
        
        
        pass
    
    
    
    
def crossEntropy_loss(y_pred, y_train):
    '''
        Computes the cross entropy loss between the predicted values and the actual values
        
        Args:
            y_pred: predicted probabilities of the categories obtained during forward pass
            y_train: actual category of the feature/ data point
    
    '''
    loss = 0
    yPred = np.array([np.array(xi) for xi in y_pred])
    #print(yPred.shape)
    #print(y_train)
    loss=(-1/y_train.shape[0])*np.sum(np.multiply(y_train,np.log(y_pred)))
    #print(loss)
    #print(loss)
    
    return loss
    pass

def accuracy(y_pred,y_train):
    '''
        Computes the accuracy between the predicted values and actual labels
    
        Args:
            y_pred: predicted probabilities of the categories obtained during forward pass
            y_train: actual category of the feature/ data point

    '''
    
    acc=0
    #y_pred = np.array([np.array(xi) for xi in y_pred])
    y_pred=np.argmax(y_pred,axis=1)
    y_train=np.argmax(y_train,axis=1)
    #print(y_train.shape)
    acc=(1/y_train.shape[0])*(np.sum(y_train == y_pred))*100
    
    
    
    return acc
    pass
        


In [3]:
# Initialize the neural network model and specify the parameters 

hidden_nodes=cols-1
nnobj= NeuralNetwork(cols,hidden_nodes,3)       
epochs = 1000
learning_rate = 1e-2
loss_history = []
epoch_history = []

# Gradient Descent
for e in range(epochs):
    yPred= nnobj.forward(X_train)
    loss = crossEntropy_loss(yPred, y_train)
    if e==0 or (e+1)%100==0:
        loss_history.append(loss)
        epoch_history.append(e+1)
    nnobj.backward(X_train, y_train,yPred, lr=learning_rate)#,lmda=lmda)
    
train_loss= loss_history[-1]#?
train_accuracy= accuracy(yPred,y_train)#?
yPred_test=nnobj.forward(X_test)
test_loss= crossEntropy_loss(yPred_test, y_test)# ?
test_accuracy= accuracy(yPred_test,y_test)#?

print("Final train_loss "+ str(train_loss))    
print("Final train_accuracy "+ str(train_accuracy))    
print("Testloss " + str(test_loss))
print("Accuracy is "+ str(test_accuracy))

        
            

Final train_loss 0.5711873220630546
Final train_accuracy 70.83333333333334
Testloss 0.592665812553456
Accuracy is 66.66666666666666
